## COCO数据载入

In [1]:
import os
import numpy as np

import torch
from torch.utils.data import Dataset
import cv2
from pycocotools.coco import COCO

In [10]:
coco_root = "F://data//COCO//"
json_file='instances_train2017.json'
name='train2017'

In [4]:
coco = COCO(coco_root + 'annotations/' + json_file)

loading annotations into memory...
Done (t=14.70s)
creating index...
index created!


In [8]:
ids = coco.getImgIds()
ids[0:4]

[391895, 522418, 184613, 318219]

根据索引获取image

In [24]:
id_ = ids[3]
img_file = os.path.join(coco_root, name,'{:012}'.format(id_) + '.jpg')
img = cv2.imread(img_file)
img.shape

(640, 556, 3)

获取annotation并生成targets

In [22]:
id_ = ids[0]
anno_ids = coco.getAnnIds(imgIds=[int(id_)], iscrowd=None)
annotations = coco.loadAnns(anno_ids)
print(annotations[0].keys(),annotations[1]['bbox'],annotations[0]['area'],annotations[0]['category_id'],sep="\n")

dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id'])
[339.88, 22.16, 153.88, 300.73]
12190.44565
4


In [ ]:
def pull_anno(self, index):
    id_ = self.ids[index]

    anno_ids = self.coco.getAnnIds(imgIds=[int(id_)], iscrowd=None)
    annotations = self.coco.loadAnns(anno_ids)

    target = []
    for anno in annotations:
        if 'bbox' in anno:
            xmin = np.max((0, anno['bbox'][0]))
            ymin = np.max((0, anno['bbox'][1]))
            xmax = xmin + anno['bbox'][2]
            ymax = ymin + anno['bbox'][3]

            if anno['area'] > 0 and xmax >= xmin and ymax >= ymin:
                label_ind = anno['category_id']
                cls_id = self.class_ids.index(label_ind)

                target.append([xmin, ymin, xmax, ymax, cls_id])  # [xmin, ymin, xmax, ymax, label_ind]
        else:
            print('No bbox !!')
    return target